In [528]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [529]:
data = pd.read_csv('/Users/cosaph/Desktop/fc servette/data_management/data.csv')
#print(data.head())

In [530]:
print(len(data['PLAYER_NAME'].unique()))

84


In [531]:
# toutes mes va 
# Accélération max,Distance [20-25]km/h,
# Distance [25-40]km/h,Distance m/min,Distance totale,Décélération max,Nb acc [4/5] ms²,
# Nb dec [-5/-4] ms²,Nombre de sprints,Vmax,Distance en sprint

data['Accélération_max'] = data['Accélération max']
data['Distance_20_25km_h'] = data['Distance [20-25]km/h']

data['Distance_2540km_h'] = data['Distance [25-40]km/h']
data['Distancemmin'] = data['Distance m/min']
data['Distance_totale'] = data['Distance totale']
data['Deceleration_max'] = data['Décélération max']
data['Nb_acc_4_5ms'] = data['Nb acc [4/5] ms²']

data['Nb_dec_5_4ms'] = data['Nb dec [-5/-4] ms²']
data['Nombre_de_sprints'] = data['Nombre de sprints']
#data['Distance_en_sprint'] = data['Distance en sprint']

In [532]:
""" # pour le min group size , on va enlever les joueurs qui ont moins de 10 matchs

group_sizes = data.groupby("PLAYER_NAME").size()
#print(group_sizes)

data = data[data["PLAYER_NAME"].isin(group_sizes[group_sizes > 22].index)]

# pour le max group size , on va enlever les joueurs qui ont plus de 50 matchs

group_sizes = data.groupby("PLAYER_NAME").size()
#print(group_sizes)

#data = data[data["PLAYER_NAME"].isin(group_sizes[group_sizes < 50].index)] """

' # pour le min group size , on va enlever les joueurs qui ont moins de 10 matchs\n\ngroup_sizes = data.groupby("PLAYER_NAME").size()\n#print(group_sizes)\n\ndata = data[data["PLAYER_NAME"].isin(group_sizes[group_sizes > 22].index)]\n\n# pour le max group size , on va enlever les joueurs qui ont plus de 50 matchs\n\ngroup_sizes = data.groupby("PLAYER_NAME").size()\n#print(group_sizes)\n\n#data = data[data["PLAYER_NAME"].isin(group_sizes[group_sizes < 50].index)] '

In [533]:

# prendre juste TEAM_ID = ERVETTE PRO
data['label'] = data['TEAM_ID'].apply(lambda x: 1 if x == 'Servette PRO' else 0)

#print(data.head())

# drop na by mean data['Accélération_max'] data['Distancemmin'] data['Accélération_max']

data = data.dropna(subset=['Accélération_max', 'Distancemmin', 'Accélération_max'])
data = data.reset_index(drop=True)

# on va utiliser statsmodels pour la régression logistique
# min group size : le nombre minimum d'observations par joueur
# max group size : le nombre maximum d'observations par joueur
# changer le csv pour avoir plus d'observations par joueur



#IndexError: index 1185 is out of bounds for axis 0 with size 1177

model_lmm = smf.mixedlm("label ~ Vmax + Distancemmin + Nombre_de_sprints + Nb_acc_4_5ms", data, groups=data["PLAYER_NAME"])
results_lmm = model_lmm.fit()
print(results_lmm.summary())



           Mixed Linear Model Regression Results
Model:              MixedLM   Dependent Variable:   label   
No. Observations:   1177      Method:               REML    
No. Groups:         77        Scale:                0.0086  
Min. group size:    1         Log-Likelihood:       893.8421
Max. group size:    53        Converged:            Yes     
Mean group size:    15.3                                    
------------------------------------------------------------
                  Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------
Intercept          0.319    0.065  4.926 0.000  0.192  0.446
Vmax              -0.004    0.001 -2.738 0.006 -0.006 -0.001
Distancemmin       0.001    0.000  4.172 0.000  0.000  0.001
Nombre_de_sprints  0.003    0.001  4.006 0.000  0.001  0.004
Nb_acc_4_5ms      -0.001    0.000 -1.908 0.056 -0.002  0.000
Group Var          0.189    0.345                           



In [534]:
## modèle logistique simple
model = smf.logit("label ~ Distancemmin + Nombre_de_sprints + Accélération_max + Nb_acc_4_5ms + Vmax  ", data)
results = model.fit()
print(results.summary())

print("AIC:", results.aic)
print("BIC:", results.bic)

Optimization terminated successfully.
         Current function value: 0.505377
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                  label   No. Observations:                 1177
Model:                          Logit   Df Residuals:                     1171
Method:                           MLE   Df Model:                            5
Date:                Mon, 17 Jun 2024   Pseudo R-squ.:                  0.2708
Time:                        10:31:53   Log-Likelihood:                -594.83
converged:                       True   LL-Null:                       -815.76
Covariance Type:            nonrobust   LLR p-value:                 2.789e-93
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept           -10.0411      1.367     -7.347      0.000     -12.720      -7.362
Distance

In [535]:
# modele linéaire 
model_lin = smf.ols("label ~ Distancemmin + Nombre_de_sprints + Accélération_max + Deceleration_max + Nb_acc_4_5ms + Nb_dec_5_4ms + Nombre_de_sprints ", data)
results_lin = model_lin.fit()
print(results_lin.summary())

                            OLS Regression Results                            
Dep. Variable:                  label   R-squared:                       0.308
Model:                            OLS   Adj. R-squared:                  0.305
Method:                 Least Squares   F-statistic:                     86.92
Date:                Mon, 17 Jun 2024   Prob (F-statistic):           3.69e-90
Time:                        10:31:53   Log-Likelihood:                -637.26
No. Observations:                1177   AIC:                             1289.
Df Residuals:                    1170   BIC:                             1324.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -0.8195      0.12

In [538]:


data_player = data[data['PLAYER_NAME'] == 'J.Simo']
# mettre les données dans le model
# voir la proba que le joueur soit pro

print(results_lmm.predict(data_player))
print('-----------------')
#print(results.predict(data_player))

print('-----------------')
#print(results_lin.predict(data_player))


582     0.306652
612     0.242340
627     0.316191
634     0.260578
638     0.288997
654     0.270659
657     0.277028
672     0.291584
678     0.270250
696     0.268128
703     0.271976
723     0.269575
726     0.272819
758     0.265520
768     0.263603
783     0.273580
805     0.271502
816     0.272269
830     0.277681
834     0.253391
857     0.294332
884     0.276587
888     0.253175
898     0.293506
919     0.258307
921     0.288422
940     0.291470
942     0.292694
964     0.280089
969     0.290416
1004    0.293816
1022    0.288644
1032    0.297734
1044    0.295723
1064    0.287137
dtype: float64
-----------------
-----------------
